**SENTIMENT ANALYSIS DATASET**

We will be using the IMDb Movie Review Dataset at https://www.tensorflow.org/tutorials/text/text_classification_rnn
This dataset is already preprocessed and has a label as either positive or negative.

In [1]:
from keras.datasets import imdb
from keras.preprocessing import sequence
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584
MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words= VOCAB_SIZE)

2025-03-19 15:37:18.473632: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-19 15:37:18.868646: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-19 15:37:23.232270: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [2]:
# MORE PREPROCESSING

# here we notice that the reviews are in different lengths. So awe need to normalize them to the same size.
# () If the review is grater than 250 words we trim off the extra words
# () If the review is less then 250 words we pad to make it equal to 250
# US keras already have a function to do that

train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

In [7]:
input_shape = train_data.shape[1:]
input_shape

(250,)

In [15]:
train_data

array([[    0,     0,     0, ...,    19,   178,    32],
       [    0,     0,     0, ...,    16,   145,    95],
       [    0,     0,     0, ...,     7,   129,   113],
       ...,
       [    0,     0,     0, ...,     4,  3586, 22459],
       [    0,     0,     0, ...,    12,     9,    23],
       [    0,     0,     0, ...,   204,   131,     9]], dtype=int32)

In [8]:
# CREATING THE MODEL

model = tf.keras.Sequential([
    tf.keras.Input(shape=input_shape),
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 250, 32)        │     2,834,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 32)             │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,843,041 (10.85 MB)

 Trainable params: 2,843,041 (10.85 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# TRAINING

model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=5, validation_split=0.2, batch_size=BATCH_SIZE)

Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 17s 50ms/step - acc: 0.6185 - loss: 0.6310 - val_acc: 0.8250 - val_loss: 0.3948
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 16s 50ms/step - acc: 0.8666 - loss: 0.3295 - val_acc: 0.8686 - val_loss: 0.3300
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 16s 50ms/step - acc: 0.9001 - loss: 0.2628 - val_acc: 0.8692 - val_loss: 0.3706
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 15s 49ms/step - acc: 0.9177 - loss: 0.2197 - val_acc: 0.8782 - val_loss: 0.3019
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 15s 48ms/step - acc: 0.9384 - loss: 0.1677 - val_acc: 0.8600 - val_loss: 0.3648


In [12]:
# EVALUATION 

results = model.evaluate(test_data, test_labels)
print(results)

782/782 ━━━━━━━━━━━━━━━━━━━━ 19s 24ms/step - acc: 0.8542 - loss: 0.3828
[0.3719801902770996, 0.8568400144577026]


In [13]:
# MAKING PREDICTIONS
# the process of encoding

word_index = imdb.get_word_index() # mapping

def encode_text(text):
    tokens = tf.keras.preprocessing.text.text_to_word_sequence(text)
    tokens = [word_index[word] if word in word_index else 0 for word in tokens]
    return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "What a great movie, absolutely amazing"
encoded = encode_text(text)
print(encoded)

1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 1s 1us/step
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
  

In [14]:
# making the prediction

def predict(text):
    encoded_text = encode_text(text)
    pred = np.zeros((1,250))
    pred[0] = encoded_text
    result =  model.predict(pred)
    if result[0] > 0.5:
        print(result[0], "POSITIVE")
    else:
        print(result[0], "NEGATIVE")
    
positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really sucked. Hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step
[0.9132095] POSITIVE
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
[0.26592636] NEGATIVE
